In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad
from anndata import AnnData
import os

from sklearn.model_selection import train_test_split


In [ ]:

dat_np = dat.X

method_names = ['logreg', 't-test', 'wilcoxon']

RNA_list = ["B intermediate kappa", "B intermediate lambda", "CD4 TEM_3", 
                 "B memory kappa", "B memory lambda", "B naive kappa", 
                 "B naive lambda", "CD4 CTL", "CD4 Naive", "CD4 TCM_1", 
                 "CD4 TCM_2", "CD4 TCM_3", "CD4 TEM_1", "CD8 Naive", "CD8 TCM_1",
                 "CD8 TCM_2", "CD8 TCM_3", "CD8 TEM_1", "CD8 TEM_2", "Treg Memory", 
                 "Treg Naive", "CD8 TEM_4", "CD8 TEM_5", "CD8 TEM_6", "CD14 Mono", 
                 "CD16 Mono", "cDC2_1", "cDC2_2", "gdT_1", "gdT_2", "gdT_3", 
                 "gdT_4", "MAIT", "NK Proliferating", "NK_1", "NK_2", "NK_3", 
                 "NK_4", "NK_CD56bright", "pDC", "Platelet"]
i = 0
for RNA_name in RNA_list:
      i = i + 1
      print(i)
      datdir = "../real2newdata/data/" + RNA_name + "_160000.csv"
      dat = sc.read_csv(datdir)
      for methodname in method_names:

            dattemp = dat[0:121323,:]

            for case_sim in range(100):
                  
                  train_data, test_data = train_test_split(dat_np, test_size=0.25)


                  dat_ADT = train_data[:, 0:10]


                  dat_y = train_data[:, 10]

                  # print(dat_y)
                  bool = (dat_y == 0)
                  dattemp.X[:,0] = 1-bool.astype(int)
                  dattemp.X[:,1:11] = dat_ADT 

                  df1 = pd.DataFrame(dattemp.X[:,1:11], columns=dat.var_names[0:10])
                  aa = pd.DataFrame(dattemp.X[:,0],columns=["cell_type"])
                  df1['cell_type'] = aa.astype("category")
                  adaaa = ad.AnnData(df1.iloc[:,0:12])

                  adaaa.obs = pd.DataFrame(df1['cell_type']).astype("category")
                  sc.tl.rank_genes_groups(adaaa, 'cell_type', rankby_abs = True, \
                        method = methodname) 


                  result = pd.DataFrame(adaaa.uns['rank_genes_groups']['names'])

                  savedir = "../scanpy_results_new2/real2_" + RNA_name + "_" + "sim" + str(case_sim+1) + "_" + methodname + ".csv"
                  result.to_csv(savedir, index=False)

      